#  Web Crawling Prject 6
## '대학교 커뮤니티 사이트 '에브리타임' 크롤링'
- 대학교 커뮤니티 사이트인 '에브리타임'의 게시판에 있는 글을 크롤링
- 분석을 위해 자유게시판의 text data 수집
- Crawling 모듈로는 Selenium을 이용한다.
- Webdriver는 Chrome을 이용한다.
## 방법

1. 에브리타임에 접속
2. 로그인 및 자유게시판 접속
3. 매 페이지마다 있는 게시글 크롤링
4. 제목, 내용, 추천 수, 댓글 수, 시간, URL 등을 수집


# 가장 효율적인 방법
- 먼저 각 게시글들의 URL을 크롤링한다.
- 이후 URL을 이용해서 각 게시글에 접속한 후에 글 스크랩

# 1. URL Collection

In [11]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time as t

#options = webdriver.ChromeOptions()
#options.add_argument('headless') 
#options.add_argument('window-size = 1920x1980')

driver = webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver.get('https://everytime.kr')

driver.find_element_by_class_name('login').find_element_by_link_text('로그인').click()

driver.find_element_by_name('userid').send_keys('---------')
driver.find_element_by_name('password').send_keys('-----------')

driver.find_element_by_class_name('submit').click()

driver.find_element_by_link_text('자유게시판').click()
driver.refresh()

URL_path = 'https://everytime.kr/377390/p/'

URL_list = []
i = 0
error = 0

while True:
    
    try:
        URL = URL_path + str(i)
        driver.get(URL)
        driver.find_element_by_id('container')
        try:
            element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.wrap.articles'))
            )
        except:
            print('도저히 페이지 누루는 부분을 못 찾네')
            drver.refresh()
        
        time.sleep(1)
        driver.find_element_by_css_selector('div.wrap.articles')
        for a in driver.find_elements_by_tag_name('article'):
            URL_list.append(a.find_element_by_tag_name('a').get_attribute('href'))
            
        i += 1
        
        if i * 20 != len(URL_list):
            i -= 1
            
    except:
        print('Page is not loaded')
        driver.refresh()
        error += 1
        if error == 20:
            break
            
    print('\r Working(%d/24000) and len of ULR_list = %d' %(i, len(URL_list)), end = '')
    if i == 24000:
        break

import pandas as pd

df = pd.DataFrame(URL_list, columns = ['URL'])
df.to_csv('everytimeURL{}.csv'.format(i), index = False, encoding = 'utf-8')

In [5]:
import pandas as pd
df = pd.read_csv('everytimeURL2182.csv', encoding = 'utf-8')
URL_list = df['URL'].tolist()
URL_list = list(set(URL_list))
len(URL_list)

43584

# 2. Extract data

In [30]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://everytime.kr')

driver.find_element_by_class_name('login').find_element_by_link_text('로그인').click()

driver.find_element_by_name('userid').send_keys('----------')
driver.find_element_by_name('password').send_keys('---------------')

driver.find_element_by_class_name('submit').click()

driver.find_element_by_link_text('자유게시판').click()
driver.refresh()

title_list = []
content_list = []
comment_list = []
vote_list = []
time_list = []
error_url = []

for i, url in enumerate(URL_list):
    try:
        driver.get(url)
        driver.find_element_by_css_selector('div.wrap.articles')
        
        try:
            element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.TAG_NAME, 'article'))
            )
        except:
            print('No loading')
            drver.refresh()
            driver.get(URL_list[i])
            error_url.append(i)
            error += 1
            if error > 100:
                break
        
        title = driver.find_element_by_css_selector('h2.large').text
        content = driver.find_element_by_css_selector('p.large').text
        comment = driver.find_element_by_css_selector('ul.status li.comment').text
        vote = driver.find_element_by_css_selector('ul.status li.vote').text
        time = driver.find_element_by_css_selector('div.profile time').get_attribute('title')
        
        title_list.append(title)
        content_list.append(content)
        comment_list.append(comment)
        vote_list.append(vote)
        time_list.append(time)
        
        title_n = len(title_list)
        content_n = len(content_list)
        comment_n = len(comment_list)
        vote_n = len(vote_list)
        time_n = len(time_list)
        
        print('\r Working(%d/43584), title:content:comment:vote:time = %d:%d:%d:%d:%d' %(i, title_n, content_n, comment_n, vote_n, time_n),
              end = '')
        
        
    except:
        print('all error')
        error_url.append(i)
        driver.quit()
        
        driver = webdriver.Chrome()
        driver.maximize_window()
        driver.get('https://everytime.kr')
        driver.find_element_by_class_name('login').find_element_by_link_text('로그인').click()
        driver.find_element_by_name('userid').send_keys('---------')
        driver.find_element_by_name('password').send_keys('-------------')
        driver.find_element_by_class_name('submit').click()
        driver.find_element_by_link_text('자유게시판').click()
        driver.refresh()                
        t.sleep(1)
        
        error += 1
        if error > 100:
            break

In [34]:
import pandas as pd

data = {'title' : title_list, 'content' : content_list, 'comment' : comment_list, 'vote' : vote_list, 'time' : time_list}
df = pd.DataFrame(data)
df.to_pickle('everytime{}.pickle'.format(i))
df.to_csv('everytime_final{}.csv'.format(i), index = False, encoding = 'utf-8')


### 각 게시글 접속
1. 각 게시글에 입장할 때 click을 통해 입장
2. 각 게시글에 입장할 때 해당 게시글의 url을 통해 입장 << 이 방법 채택<br>
(이유 : click을 이용하기 위해서는 window창을 각 게시글로 이동시켜줘야 함, scrolled into view, 번거로움)

In [ ]:
%%time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time as t


options = webdriver.Chromeoptions()
options.add_argument('headless') 
options.add_argument('window-size = 1920x1980')


driver = webdriver.Chrome('chromedriver.exe')

driver.get('https://everytime.kr')

driver.find_element_by_class_name('login').find_element_by_link_text('로그인').click()

driver.find_element_by_name('userid').send_keys('-------')
driver.find_element_by_name('password').send_keys('--------')

driver.find_element_by_class_name('submit').click()

driver.find_element_by_link_text('자유게시판').click()
driver.refresh()
url = []
time = []
title = []
content = []
vote = []
comment = []
n = 0
for page_number in range(1,501,1):
    page_url = driver.current_url
    driver.implicitly_wait(3)
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    driver.execute_script('window.scrollTo(0,0);')
    post_list = driver.find_elements_by_tag_name('article')
    
    for i in range(20):         
        
        
        post_url = post_list[i].find_element_by_tag_name('a').get_attribute('href')
        
        #post_list[i].click()
        #url.append(driver.current_url)
        url.append(post_url)
        time.append(driver.find_element_by_tag_name('time').get_attribute('title'))
        
        driver.get(post_url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html5lib')
        print('start')
        title.append(soup.select('h2.large')[0].text.strip())
        print(i)
        content.append(soup.select('p.large')[0].text.strip())
        vote.append(soup.select('li.vote')[0].text.strip())
        comment.append(soup.select('li.comment')[0].text.strip())
        driver.back()
        
    if page_number % 50 == 0:
        df = pd.DataFrame({'title' : title, 'content' : content, 'vote' : vote, 'comment' : comment, 'url' : url, 'time' : time})
        df = df[['title', 'content', 'comment', 'vote', 'time', 'url']]
        df.to_csv('everytime{}.csv'.format(page_number // 50))
        url = []
        time = []
        title = []
        content = []
        vote = []
        comment = []
    driver.implicitly_wait(20)
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    driver.find_element_by_css_selector('a.next').click()

### BeautifulSoup 파싱 없이 바로
- driver에서 element로 바로 찾아서 append
- 파싱으로 인해 페이지의 로드 시간이 소요되어 가끔 뒤에 코드가 먼저 실행되는 문제 해결
- 시간 단축

###  추가사항
- 속도 개선을 위해 절대적 대기(sleep) -> 명시적 대기(explicit wait)로 변경
- 속도 개선을 위해 chrome driver의 이미지 출력 옵션을 제거

In [ ]:
%%time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time as t

# 명시적 대기
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size = 1920x1980')


driver = webdriver.Chrome('chromedriver.exe')

driver.get('https://everytime.kr')

driver.find_element_by_class_name('login').find_element_by_link_text('로그인').click()

driver.find_element_by_name('userid').send_keys('------------')
driver.find_element_by_name('password').send_keys('------------')

driver.find_element_by_class_name('submit').click()

driver.find_element_by_link_text('자유게시판').click()
driver.refresh()

url = []
time = []
title = []
content = []
vote = []
comment = []
n = 0

#for page_number in range(1,501,1):
for page_number in range(2):
    page_url = driver.current_url

    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    driver.execute_script('window.scrollTo(0,0);')
    t.sleep(1)
    post_url_list = [u.get_attribute('href') for u in driver.find_elements_by_css_selector('article a.article')]
    for i in range(3):
        #post_list = driver.find_elements_by_tag_name('article')
        #post_url = post_list[i].find_element_by_tag_name('a').get_attribute('href')
        post_url = post_url_list[i]
        #post_list[i].click()
        driver.get(post_url)
        
        t.sleep(2)
        url.append(post_url)
        time.append(driver.find_element_by_tag_name('time').get_attribute('title'))


        # method1
        title.append(driver.find_element_by_css_selector('article a.article h2.large').text)

        content.append(driver.find_element_by_css_selector('article a.article p.large').text.strip())
        vote.append(driver.find_element_by_css_selector('article a.article ul.status li.vote').text)
        comment.append(driver.find_element_by_css_selector('article a.article ul.status li.comment').text)

        driver.back()
        
    if page_number % 50 == 0:
        df = pd.DataFrame({'title' : title, 'content' : content, 'vote' : vote, 'comment' : comment, 'url' : url, 'time' : time})
        df = df[['title', 'content', 'comment', 'vote', 'time', 'url']]
        df.to_csv('everytime_data/everytime{}.csv'.format(page_number))
        print(page_number)
        title, content, vote, comment, url, time = [], [], [], [], [], []
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    t.sleep(2)
    driver.find_element_by_css_selector('a.next').click()

### 중간부터 시작
- 여러 이유로 반복문이 중단되었을 때 이용
- 마지막 page number을 입력해서 해당 페이지부터 재시작

In [ ]:
%%time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time as t


options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size = 1920x1980')


driver = webdriver.Chrome('chromedriver.exe')

driver.get('https://everytime.kr')

driver.find_element_by_class_name('login').find_element_by_link_text('로그인').click()

driver.find_element_by_name('userid').send_keys('-----------')
driver.find_element_by_name('password').send_keys('-------------')

driver.find_element_by_class_name('submit').click()

page_n = 46 # The page number to start
driver.get('https://everytime.kr/377390/p/' + str(page_n))

url = []
time = []
title = []
content = []
vote = []
comment = []
n = 0

for page_number in range(page_n,2062,1):
    
    page_url = driver.current_url

    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    driver.execute_script('window.scrollTo(0,0);')
    t.sleep(1)
    post_url_list = [u.get_attribute('href') for u in driver.find_elements_by_css_selector('article a.article')]
    for i in range(20):
        #post_list = driver.find_elements_by_tag_name('article')
        #post_url = post_list[i].find_element_by_tag_name('a').get_attribute('href')
        post_url = post_url_list[i]
        #post_list[i].click()
        driver.get(post_url)
        
        t.sleep(2)
        url.append(post_url)
        time.append(driver.find_element_by_tag_name('time').get_attribute('title'))


        # method1
        title.append(driver.find_element_by_css_selector('article a.article h2.large').text)

        content.append(driver.find_element_by_css_selector('article a.article p.large').text.strip())
        vote.append(driver.find_element_by_css_selector('article a.article ul.status li.vote').text)
        comment.append(driver.find_element_by_css_selector('article a.article ul.status li.comment').text)

        driver.back()
        
    if page_number % 50 == 0:
        df = pd.DataFrame({'title' : title, 'content' : content, 'vote' : vote, 'comment' : comment, 'url' : url, 'time' : time})
        df = df[['title', 'content', 'comment', 'vote', 'time', 'url']]
        df.to_csv('everytime_data/everytime{}.csv'.format(page_number))
        print(page_number)
        title, content, vote, comment, url, time = [], [], [], [], [], []
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    t.sleep(2)
    driver.find_element_by_css_selector('a.next').click()

## Concat csv files
- 50page마다 저장했던 크롤링 데이터를 concat

In [4]:
import glob
import pandas as pd
path = 'everytime_data/clean_data'
all_files = glob.glob(path + '/*.csv')
print(all_files, '\n')
df_list = [pd.read_csv(file, encoding = 'utf-8', index_col = None) for file in all_files]
df = pd.concat(df_list)

df.drop_duplicates('title', keep = 'first', inplace = True)
df.reset_index(drop = True, inplace = True)
df = df[['title', 'content', 'comment', 'vote', 'time', 'url']]

df.to_csv('everytime_data/clean_data/everytime.csv', encoding = 'utf-8', index = False)
print(df.info())
df.head()

['everytime_data/clean_data\\everytime.csv', 'everytime_data/clean_data\\everytime100.csv', 'everytime_data/clean_data\\everytime150.csv', 'everytime_data/clean_data\\everytime200.csv', 'everytime_data/clean_data\\everytime240.csv', 'everytime_data/clean_data\\everytime250.csv', 'everytime_data/clean_data\\everytime300.csv', 'everytime_data/clean_data\\everytime350.csv', 'everytime_data/clean_data\\everytime400.csv', 'everytime_data/clean_data\\everytime425.csv', 'everytime_data/clean_data\\everytime450.csv', 'everytime_data/clean_data\\everytime46.csv', 'everytime_data/clean_data\\everytime50.csv', 'everytime_data/clean_data\\everytime500.csv', 'everytime_data/clean_data\\everytime550.csv', 'everytime_data/clean_data\\everytime570.csv', 'everytime_data/clean_data\\everytime600.csv', 'everytime_data/clean_data\\everytime650.csv', 'everytime_data/clean_data\\everytime700.csv'] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12939 entries, 0 to 12938
Data columns (total 6 columns):
t

,title,content,comment,vote,time,url
0,지하철에서 뭐 좀 쳐먹지마 제발,냄새 역겨워,16,0,2018-11-01 12:56:28,https://everytime.kr/377390/v/56050760
1,계절학기 수요 조사 신청 좀 도와주세요ㅠㅠ,중선인 교양이라 주관학과가 대양휴머니티칼리지인데요ㅠㅠㅠ 학교 홈페이지에 치니까 행정...,0,0,2018-11-01 12:55:30,https://everytime.kr/377390/v/56050712
2,눈떠보니,11정거장이나 지나있네?,8,0,2018-11-01 12:11:09,https://everytime.kr/377390/v/56048337
3,휴게실에서 소리틀어놓고 스포츠중계보는건 뭐하자는거지,딴엔 소리 작게 틀어놨다고 생각하는모양인데 ㅋㅋ;,4,1,2018-11-01 11:47:51,https://everytime.kr/377390/v/56046985
4,와 방금기상,오늘 1교시부터 풀수업인데...\r\r\r\r\r\r\r\r\r\r\r\r\n이참에...,5,1,2018-11-01 11:44:22,https://everytime.kr/377390/v/56046827
